In [23]:
import math

import numpy as np
import sympy as sp
import pandas as pd
from scipy import integrate

# https://pomax.github.io/bezierinfo/legendre-gauss.html
GaussTable = [[[0], [2]],
              [[-1/np.sqrt(3), 1/np.sqrt(3)], [1, 1]], 
              [[-np.sqrt(3/5), 0, np.sqrt(3/5)], [5/9, 8/9, 5/9]], 
              [[-0.861136, -0.339981, 0.339981, 0.861136], [0.347855, 0.652145, 0.652145, 0.347855]], 
              [[-0.90618, -0.538469, 0, 0.538469, 0.90618], [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]], 
              [[-0.93247, -0.661209, -0.238619, 0.238619, 0.661209, 0.93247], [0.171324, 0.360762, 0.467914, 0.467914, 0.360762,  0.171324]],
              [[0.94910, -0.94910, 0.741531, -0.74153, -0.40584, 0.40584, 0], [0.12948, 0.129484, 0.27970, 0.27970, 0.38183, 0.38183, 0.41795]],
              [[0.96028, -0.96028, 0.79666, -0.79666, 0.52553, -0.52553, 0.18343, -0.18343], [0.10122, 0.10122, 0.22238, 0.22238, 0.31370, 0.31370, 0.36268, 0.36268]],
              [[0.613371, -0.61337, 0.32425, -0.32425, 0.96816, -0.96816, 0.83603, -0.83603, 0],[0.26061, 0.26061, 0.31234, 0.31234, 0.08127, 0.08127, 0.18064, 0.18064, 0.33023]],
              [[0.97390, -0.97390, 0.86506, -0.86506, 0.67940, -0.67940, 0.43339, -0.43339, 0.14887, -0.14887], [0.06667, 0.06667, 0.14945, 0.14945, 0.21908, 0.21908, 0.26926, 0.26926, 0.29552, 0.29552]],
              [[0.97822, -0.97822, 0.88706, -0.88706, 0.73015, -0.73015, 0.51909, -0.51909, 0.26954, -0.26954, 0],[0.05566, 0.05566, 0.12558, 0.12558, 0.18629, 0.18629, 0.23319, 0.23319, 0.26280, 0.26280, 0.27292]],
              [[0.98156, -0.98156, 0.90411, -0.90411, 0.76990, -0.76990, 0.58731, -0.58731, 0.36783, -0.36783, 0.12523, -0.12523], [0.04717, 0.04717, 0.10693, 0.10693, 0.16007, 0.16007, 0.20316, 0.20316, 0.23349, 0.23349, 0.24914, 0.24914]],
              [[0.98418, -0.98418, 0.91759, -0.91759, 0.80157, -0.80157, 0.64234, -0.64234, 0.44849, -0.44849, 0.23045, -0.23045, 0], [0.04048, 0.04048, 0.09212, 0.09212, 0.13887, 0.13887, 0.17814, 0.17814, 0.20781, 0.20781, 0.22628, 0.22628, 0.23255]],
              [[0.98628, -0.98628, 0.92843, -0.92843, 0.82720, -0.82720, 0.68729, -0.68729, 0.51524, -0.51524, 0.31911, -0.31911, 0.10805, -0.10805], [0.03511, 0.03511, 0.08015, 0.08015, 0.12151, 0.12151, 0.15720, 0.15720, 0.18553, 0.18553, 0.20519, 0.20519, 0.21526, 0.21526]],
              [[0.98799, -0.98799, 0.93727, -0.93727, 0.84820, -0.84820, 0.72441, -0.72441, 0.57097, -0.57097, 0.39415, -0.39415, 0.20119, -0.20119, 0], [0.03075, 0.03075, 0.07036, 0.07036, 0.10715, 0.10715, 0.13957, 0.13957, 0.16626, 0.16626, 0.18616, 0.18616, 0.19843, 0.19843, 0.20257]],
              [[0.98940, -0.98940, 0.94457, -0.94457, 0.86563, -0.86563, 0.75540, -0.75540, 0.61787, -0.61787, 0.45801, -0.45801, 0.28160, -0.28160, 0.09501, -0.09501], [0.02715, 0.02715, 0.06225, 0.06225, 0.09515, 0.09515, 0.12462, 0.12462, 0.14959, 0.14959, 0.16915, 0.16915, 0.18260, 0.18260, 0.18945, 0.18945]],
              [[0.99057, -0.99057, 0.95067, -0.95067, 0.88023, -0.88023, 0.78151, -0.78151, 0.65767, -0.65767, 0.51269, -0.51269, 0.35123, -0.35123, 0.17848, -0.17848, 0], [0.02414, 0.02414, 0.05545, 0.05545, 0.08503, 0.08503, 0.11188, 0.11188, 0.13513, 0.13513, 0.15404, 0.15404, 0.16800, 0.16800, 0.17656, 0.17656, 0.17944]]
              ]

display(pd.DataFrame(GaussTable, columns=["Integration Points", "Corresponding Weights"]))

def IGAL(f, n, a, b):
  n = int(n)
  return sum([(b - a)/2*GaussTable[n - 1][1][i]*f((b - a)/2*(GaussTable[n - 1][0][i] + 1) + a) for i in range(n)])

def f(x): return x**2 * math.sqrt(1 - x ** 2)

lo = 0
hi = 1
Iexact, error = integrate.quad(f, lo, hi)
print("Iexact: ",Iexact)
Itable = [[i + 1, sp.N(IGAL(f, i + 1, lo, hi)), (Iexact - IGAL(f, i + 1, lo, hi))/Iexact] for i in range(len(GaussTable))]
Itable = pd.DataFrame(Itable, columns=["Number of Integration Points",  "Numerical Integration Results", "Relative Error"])

display(Itable)

,Integration Points,Corresponding Weights
0,[0],[2]
1,"[-0.5773502691896258, 0.5773502691896258]","[1, 1]"
2,"[-0.7745966692414834, 0, 0.7745966692414834]","[0.5555555555555556, 0.8888888888888888, 0.555..."
3,"[-0.861136, -0.339981, 0.339981, 0.861136]","[0.347855, 0.652145, 0.652145, 0.347855]"
4,"[-0.90618, -0.538469, 0, 0.538469, 0.90618]","[0.236927, 0.478629, 0.568889, 0.478629, 0.236..."
5,"[-0.93247, -0.661209, -0.238619, 0.238619, 0.6...","[0.171324, 0.360762, 0.467914, 0.467914, 0.360..."
6,"[0.9491, -0.9491, 0.741531, -0.74153, -0.40584...","[0.12948, 0.129484, 0.2797, 0.2797, 0.38183, 0..."
7,"[0.96028, -0.96028, 0.79666, -0.79666, 0.52553...","[0.10122, 0.10122, 0.22238, 0.22238, 0.3137, 0..."
8,"[0.613371, -0.61337, 0.32425, -0.32425, 0.9681...","[0.26061, 0.26061, 0.31234, 0.31234, 0.08127, ..."
9,"[0.9739, -0.9739, 0.86506, -0.86506, 0.6794, -...","[0.06667, 0.06667, 0.14945, 0.14945, 0.21908, ..."


Iexact:  0.196349540849358


,Number of Integration Points,Numerical Integration Results,Relative Error
0,1,0.216506350946110,-0.102658
1,2,0.213033378945778,-0.084970
2,3,0.200591575862339,-0.021605
3,4,0.198161405916995,-0.009228
4,5,0.197299263167200,-0.004837
5,6,0.196911132063262,-0.002860
6,7,0.196708184625368,-0.001827
7,8,0.196592852163062,-0.001239
8,9,0.196519558648114,-0.000866
9,10,0.196475156876125,-0.000640
